# WIP: Back-test

This example is still WIP.
* Load Data
* Back Test - the Basic Idea

In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-10 20:01:12,058 INFO Keys of d_instrument_data: dict_keys(['EEMA', 'IEV', 'IVV', 'VPL'])
2022-03-10 20:01:12,067 INFO IVV (SP500 ETF): 
        date       open       high        low      close  close_adj   volume
0 2000-05-19  142.65625  142.65625  140.25000  140.68750  94.121216   775500
1 2000-05-22  140.59375  140.59375  136.81250  139.81250  93.535789  1850600
2 2000-05-23  140.21875  140.21875  137.68750  137.68750  92.114151   373900
3 2000-05-24  137.75000  140.06250  136.65625  139.75000  93.494003   400300
4 2000-05-25  140.03125  140.93750  137.87500  138.46875  92.636810    69600



### Back Test - the Basic Idea

WIP
* back test is to understand how your strategy behaves under various scenanios, not to prove your strategy can beat xxx.
* most investment products are not built to beat xxx.
* historical performance cannot represent future performance.


### Back Test - WIP

In [5]:
from grandma_stock_valuation.back_test import GrandmaBackTester

backtester = GrandmaBackTester(
    backtest_years=10,
    adjust_freq_months=1,
    init_parameters={'recent_months':0, 'train_years':10, 'min_train_years':5, 'date_end':None},
    fit_parameters={'price_col':'close_adj', 'log':True, 'n_std':1.5},
    valuate_parameters={'min_annual_return':0.01},
    allocation_parameters={'transformation':'sigmoid', 'scale':1},
    with_cash=True,
    with_correlation_weights=True,
    verbose=1,
    printfunc=logPrint
)

backtester.runBackTest(d_instrument_data)

backtester.df_average_value


2022-03-10 20:01:59,612 INFO To backtest 10 years, from 2012-03-01 to 2022-02-28
2022-03-10 20:01:59,614 INFO EEMA's start date 2012-02-09 is beyond 2007-03-01 for full back-test.
2022-03-10 20:02:00,134 INFO Adjust portfolio on 2012-04-02, total value = 1.004962
2022-03-10 20:02:00,450 INFO Adjust portfolio on 2012-05-01, total value = 0.988057
2022-03-10 20:02:00,803 INFO Adjust portfolio on 2012-06-01, total value = 0.906618
2022-03-10 20:02:01,158 INFO Adjust portfolio on 2012-07-02, total value = 0.969176
2022-03-10 20:02:01,504 INFO Adjust portfolio on 2012-08-01, total value = 0.969958
2022-03-10 20:02:01,857 INFO Adjust portfolio on 2012-09-04, total value = 0.993194
2022-03-10 20:02:02,177 INFO Adjust portfolio on 2012-10-01, total value = 1.017008
2022-03-10 20:02:02,511 INFO Adjust portfolio on 2012-11-01, total value = 1.025631
2022-03-10 20:02:02,856 INFO Adjust portfolio on 2012-12-03, total value = 1.031676
2022-03-10 20:02:03,200 INFO Adjust portfolio on 2013-01-02, tot

In [11]:
import pandas as pd

df_average_value = pd.concat(backtester.d_portfolio.values()).groupby('ticker')['current_value'].mean().reset_index()
df_average_value['avg_pct_allocation'] = df_average_value['current_value'] / df_average_value['current_value'].sum()
df_average_value = df_average_value[['ticker', 'avg_pct_allocation']]

df_average_value


,ticker,avg_pct_allocation
0,EEMA,0.134615
1,IEV,0.182015
2,IVV,0.194760
3,VPL,0.082779
4,cash,0.405831


In [10]:
df_instrument_growth = backtester._df_instrument_prices.iloc[backtester._index_start:].copy()
d_first_prices = df_instrument_growth.apply(lambda se: se.dropna().iloc[0])
df_instrument_growth.apply(lambda se: se/d_first_prices[se.name])

df_instrument_growth.head()

,price_EEMA,price_IEV,price_IVV,price_VPL
date,,,,
2012-03-01,45.518845,28.437004,114.176628,40.559399
2012-03-02,46.165085,28.114273,113.879234,40.223755
2012-03-05,45.576843,28.001698,113.408363,39.964397
2012-03-06,45.576843,26.883429,111.657005,39.209198
2012-03-07,44.193214,27.266191,112.491371,39.514324


,price_EEMA,price_IEV,price_IVV,price_VPL
date,,,,
2012-03-01,1.000000,1.000000,1.000000,1.000000
2012-03-02,1.014197,0.988651,0.997395,0.991725
2012-03-05,1.001274,0.984692,0.993271,0.985330
2012-03-06,1.001274,0.945368,0.977932,0.966711
2012-03-07,0.970877,0.958828,0.985240,0.974233
